In [2]:
# Necessary libs
import feather # import data from file
import pandas as pd, numpy as np # data structures and analysis

import matplotlib.pyplot as plt # graphical plotting
plt.style.use('ggplot') # Pretty style

/Users/matt/projects/d4d/drug-spending/.env/lib/python3.6/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')
/Users/matt/projects/d4d/drug-spending/.env/lib/python3.6/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [5]:
# Load up drug names data
df_drugnames = feather.read_dataframe('../data/drugnames.feather')

In [6]:
# Load up spending data for each available year
year_file_map = { year: '../data/spending-' + str(year) + '.feather' for year in range(2011, 2016) }
df_years = { year: feather.read_dataframe(year_file_map[year]) for year in year_file_map }

In [42]:
# Set the index of each DataFrame to be drug names, then remove those columns
for year in df_years:
    index_values = df_years[year].iloc[:, :2].values.T
    df_years[year].index = pd.MultiIndex.from_arrays(index_values)
    df_years[year] = df_years[year].iloc[:, 2:]

In [43]:
# Create a Panel from all the individual year data
drug_spending = pd.Panel(df_years)
drug_spending

<class 'pandas.core.panel.Panel'>
Dimensions: 5 (items) x 4498 (major_axis) x 10 (minor_axis)
Items axis: 2011 to 2015
Major_axis axis: (10 WASH , SULFACETAMIDE SODIUM ) to (ZYVOX , LINEZOLID )
Minor_axis axis: claim_count to out_of_pocket_avg_lowincome

In [44]:
# Top 10 drugs by total cumulative dollars spent, 2011-2015, descending (in billions of dollars)
(drug_spending.loc[:, :, 'total_spending'].sum().dropna().sort_values(ascending=False) / 1000000000) \
    .head(10) \
    .map(lambda x: '$ ' + ('%.1fB' % x).rjust(5, ' '))

LANTUS/LANTUS SOLOSTAR   INSULIN GLARGINE,HUM.REC.ANLOG     $ 14.2B
NEXIUM                   ESOMEPRAZOLE MAGNESIUM             $ 11.3B
CRESTOR                  ROSUVASTATIN CALCIUM               $ 10.8B
ADVAIR DISKUS            FLUTICASONE/SALMETEROL             $ 10.4B
ABILIFY                  ARIPIPRAZOLE                       $  9.4B
SPIRIVA                  TIOTROPIUM BROMIDE                 $  9.2B
HARVONI                  LEDIPASVIR/SOFOSBUVIR              $  7.7B
JANUVIA                  SITAGLIPTIN PHOSPHATE              $  7.3B
REVLIMID                 LENALIDOMIDE                       $  6.9B
NAMENDA                  MEMANTINE HCL                      $  6.0B
dtype: object

In [45]:
# Top 10 drugs by total cumulative claims, 2011-2015, descending (in millions of claims)
(drug_spending.loc[:, :, 'claim_count'].sum().dropna().sort_values(ascending=False) / 1000000) \
    .head(10) \
    .map(lambda x: '%.1fM' % x)

SIMVASTATIN                 SIMVASTATIN                   181.3M
LISINOPRIL                  LISINOPRIL                    181.1M
LEVOTHYROXINE SODIUM        LEVOTHYROXINE SODIUM          169.9M
AMLODIPINE BESYLATE         AMLODIPINE BESYLATE           167.6M
HYDROCODONE-ACETAMINOPHEN   HYDROCODONE/ACETAMINOPHEN     162.0M
OMEPRAZOLE                  OMEPRAZOLE                    155.9M
FUROSEMIDE                  FUROSEMIDE                    129.2M
ATORVASTATIN CALCIUM        ATORVASTATIN CALCIUM          114.8M
METFORMIN HCL               METFORMIN HCL                 109.0M
METOPROLOL TARTRATE         METOPROLOL TARTRATE           102.1M
dtype: object

In [48]:
# Find claim counts and total spending in 2011 for all forms of omeprazole (Nexium)
all_omeprazoles = ['OMEPRAZOLE' in generic for (brand, generic) in drug_spending[2011].index.values]
drug_spending[2011][all_omeprazoles][['claim_count', 'total_spending']].dropna().sort_values(by='claim_count', ascending=False)

claim_count  \
OMEPRAZOLE                     OMEPRAZOLE                       27360685.0   
NEXIUM                         ESOMEPRAZOLE MAGNESIUM            8224024.0   
OMEPRAZOLE-SODIUM BICARBONATE  OMEPRAZOLE/SODIUM BICARBONATE       71956.0   
VIMOVO                         NAPROXEN/ESOMEPRAZOLE MAG           34216.0   
ZEGERID                        OMEPRAZOLE/SODIUM BICARBONATE       25275.0   
PRILOSEC                       OMEPRAZOLE                          10950.0   
                               OMEPRAZOLE MAGNESIUM                 2056.0   
NEXIUM I.V.                    ESOMEPRAZOLE SODIUM                   704.0   

                                                               total_spending  
OMEPRAZOLE                     OMEPRAZOLE                        6.768475e+08  
NEXIUM                         ESOMEPRAZOLE MAGNESIUM            1.970138e+09  
OMEPRAZOLE-SODIUM BICARBONATE  OMEPRAZOLE/SODIUM BICARBONATE     1.547595e+07  
VIMOVO                         NAPROXEN/ESOMEPRAZOLE MAG         3.201527e+06  
ZEGERID                        OMEPRAZOLE/SODIUM BICARBONATE     7.420869e+06  
PRILOSEC                       OMEPRAZOLE                        4.080108e+06  
                               OMEPRAZOLE MAGNESIUM              6.260247e+05  
NEXIUM I.V.                    ESOMEPRAZOLE SODIUM               2.164515e+05